In [5]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 59.9 MB/s eta 0:00:00


In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
from itertools import zip_longest
import time

def get_reviews(url): #returns list
    comments_output = []
    result = requests.get(url)
    src = result.content
    soup = BeautifulSoup(src, 'lxml')

    comments = soup.find_all("div", {"class": "description"})

    for i in range(len(comments)):
        comments_output.append(comments[i].text)

    return comments_output

In [2]:
#data scrapping
import os
# Check if the CSV file already exists
if os.path.exists('CSV_OUT.csv'):
    df = pd.read_csv('CSV_OUT.csv')  # Load existing CSV file into DataFrame
else:
    df = pd.DataFrame()  # Create an empty DataFrame with a 'Review' column

for i in range(13): #13 review page
    time.sleep(1)
    reviews = get_reviews("https://www.chickadvisor.com/item/quality-street-chocolates/?page={0}#reviews".format(i+1))
    df = df.append(pd.DataFrame({'Review': reviews}))
    print(f"Page {i+1} has been scrapped successfully")

df = df.reset_index(drop=True)
df.to_csv('CSV_OUT.csv')

<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


Page 1 has been scrapped successfully


<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


Page 2 has been scrapped successfully


<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


...

<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


Page 11 has been scrapped successfully


<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


Page 12 has been scrapped successfully
Page 13 has been scrapped successfully


<ipython-input-2-6d77fe32df47>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(pd.DataFrame({'Review': reviews}))


In [4]:
for index, _ in df.iterrows():
    pass
print(index)

129


In [6]:
from transformers import pipeline
sentimentanalyzer = pipeline("sentiment-analysis")
aspectclassifier = pipeline("zero-shot-classification")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [7]:
#aspect:rank
aspects = {
            'Taste':[0,0], 'Packaging':[0,0], 'Price':[0,0]
}
products_aspect_dict = {}

for aspect in aspects:
    df[f'{aspect}POS'] = 0
    df[f'{aspect}NEG'] = 0


In [ ]:
for index, row in df.iterrows():
        reviewId = index
        aspect_out = aspectclassifier(row['Review'][:512], candidate_labels=list(aspects.keys()))
        '''
        aspect_out INPUT OUTPUT SAMPLE:
        sequence = "Who are you voting for in 2020?"
        candidate_labels = ["politics", "public health", "economics"]

        classifier(sequence, candidate_labels)
        --------------------------------------------------------------
        {'labels': ['politics', 'economics', 'public health'],
         'scores': [0.972518801689148, 0.01458414364606142, 0.012897025793790817],
         'sequence': 'Who are you voting for in 2020?'}
        '''
        sentiment_out = sentimentanalyzer(row['Review'][:512])
        '''
        sentiment_out INPUT OUTPUT SAMPLE:
        sentiment_pipeline = pipeline("sentiment-analysis")
        data = ["I love you", "I hate you"]
        ----------------------------------------------------
        [{'label': 'POSITIVE', 'score': 0.9998},
         {'label': 'NEGATIVE', 'score': 0.9991}]
        '''
        print(f"Review ID: {reviewId}")
        print(f"Text: {row['Review'][:512]}")
        print(f"Dictionary: {sentiment_out}")

        for aspect, aspect_score in zip(aspect_out['labels'], aspect_out['scores']):
            if sentiment_out[0]['label'] == 'POSITIVE':
                df.loc[index, f'{aspect}POS'] = sentiment_out[0]['score'] * aspect_score
                df.loc[index, f'{aspect}NEG'] = 0

            elif sentiment_out[0]['label'] == 'NEGATIVE':
                df.loc[index, f'{aspect}POS'] = 0
                df.loc[index, f'{aspect}NEG'] = sentiment_out[0]['score'] * aspect_score


            print(f"Aspect: {aspect}, aspect_scores: {aspect_score}")

        df.to_csv('CSV_OUT.csv')
        print('-'*100)



Review ID: 0
Text: 
Low quality Very low quality in all aspects. The chocolate tasted like cooking chocolate. Flavours weren’t the best/a bit bland, and why do they have names like “the purple one”? That’s not descriptive as to how it tastes? Don’t like the dark purple plastic hexagon they come in either. Needs a complete remodel

Dictionary: [{'label': 'NEGATIVE', 'score': 0.9995036125183105}]
Aspect: Taste, aspect_scores: 0.5899762511253357
Aspect: Price, aspect_scores: 0.23080356419086456
Aspect: Packaging, aspect_scores: 0.17922024428844452
----------------------------------------------------------------------------------------------------
